In [ ]:
pip install gradio

In [ ]:
import gradio as gr
from transformers import pipeline
import re

# Initialize the model
model_name = "sarvamai/sarvam-2b-v0.5"
pipe = pipeline("text-generation", model=model_name, device=0)

# Supported languages
LANGUAGES = ["English", "Bengali", "Gujarati", "Hindi", "Kannada", "Malayalam", "Marathi", "Oriya", "Punjabi", "Tamil", "Telugu"]

def is_relevant_response(query, response):
    query_words = set(re.findall(r'\w+', query.lower()))
    response_words = set(re.findall(r'\w+', response.lower()))
    return len(query_words.intersection(response_words)) > 0

def chatbot(message, history, language):
    # Prepare the prompt with more context and instructions
    prompt = f"""This is a conversation in {language}.
The AI assistant is helpful, polite, and provides relevant responses to the user's messages.
Always respond in the language specified.

Conversation history:
"""
    for human, ai in history:
        prompt += f"Human: {human}\nAI: {ai}\n"
    prompt += f"Human: {message}\nAI: "

    try:
        # Generate multiple responses and choose the most relevant one
        responses = pipe(prompt, max_new_tokens=150, temperature=0.7, repetition_penalty=1.2, top_k=50, num_return_sequences=3)

        for response in responses:
            ai_response = response['generated_text'].split("AI:")[-1].strip()
            if is_relevant_response(message, ai_response):
                return ai_response

        # If no relevant response is found
        return f"I apologize, but I'm having trouble generating a relevant response in {language}. Could you please rephrase your message?"

    except Exception as e:
        return f"An error occurred: {str(e)}. Please try again."

# Create the Gradio interface
iface = gr.ChatInterface(
    chatbot,
    additional_inputs=[
        gr.Dropdown(choices=LANGUAGES, label="Select Language", value="English")
    ],
    title="Refined Multilingual Indian Chatbot",
    description="Chat in multiple Indian languages using the sarvam-2b model. The chatbot aims to provide relevant responses based on the selected language.",
    examples=[
        ["Hello, how are you?", "English"],
        ["नमस्ते, आप कैसे हैं?", "Hindi"],
        ["வணக்கம், எப்படி இருக்கிறீர்கள்?", "Tamil"],
    ],
    theme="soft"
)

# Launch the interface
iface.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bc1f1ca38f1f040e5b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
